In [4]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.neighbors import NearestNeighbors

In [2]:
df = pd.read_csv('homework_6.1.csv')

# Display the first few rows and check column names
df.head()

,Unnamed: 0,Z,X,Y
0,0,0.548814,0,-0.823220
1,1,0.715189,1,0.842405
2,2,0.602763,1,0.898618
3,3,0.544883,0,-0.817325
4,4,0.423655,0,-0.635482


In [5]:
treated = df[df['X'] == 1].reset_index(drop=True)
untreated = df[df['X'] == 0].reset_index(drop=True)

# Nearest neighbor for each treated unit (for ATT and overall ATE)
nn_untreated = NearestNeighbors(n_neighbors=1).fit(untreated[['Z']])
dist_treated, idx_treated = nn_untreated.kneighbors(treated[['Z']])
counterfactuals_treated = untreated.iloc[idx_treated.flatten()].reset_index(drop=True)

# Nearest neighbor for each untreated unit (for ATU and marginal treatment effect)
nn_treated = NearestNeighbors(n_neighbors=1).fit(treated[['Z']])
dist_untreated, idx_untreated = nn_treated.kneighbors(untreated[['Z']])
counterfactuals_untreated = treated.iloc[idx_untreated.flatten()].reset_index(drop=True)

# Calculate treatment effects
# ATE: All treated + all untreated, matched to the other group
treated_effects = treated['Y'].values - counterfactuals_treated['Y'].values
untreated_effects = counterfactuals_untreated['Y'].values - untreated['Y'].values
ate = np.mean(np.concatenate([treated_effects, untreated_effects]))

# ATT: Only treated matched to untreated
att = np.mean(treated_effects)

# ATU: Only untreated matched to treated
atu = np.mean(untreated_effects)

# Marginal Treatment Effect: Max treatment effect for untreated items
mte = np.max(counterfactuals_untreated['Y'].values - untreated['Y'].values)

ate, att, atu, mte

(np.float64(1.6952701427487222),
 np.float64(1.846408507179443),
 np.float64(1.5494765534845105),
 np.float64(2.1724698851117035))